In [ ]:
# PyData libraries
import numpy as np
from numba import njit
from scipy.spatial.distance import squareform

# Persistence tools
from gtda.homology import VietorisRipsPersistence
from steenroder import barcodes, rips_barcodes
from steenroder.plotting import plot_diagrams

# Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.style'] = "normal"
matplotlib.rcParams['font.variant'] = "normal"
matplotlib.rcParams['font.serif'] = "Computer Modern Roman"

# Construct the flat Klein bottle

In [ ]:
def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

In [ ]:
num = 10

print(f"The total number of vertices is {num**2}")

In [ ]:
square = cartesian_product(np.linspace(0, 1, num=num, endpoint=False), np.linspace(0, 1, num=num, endpoint=False))
n = len(square)

squares = np.tile(square.T, 9).T

squares[n:2 * n] += [0, 1]

squares[2 * n:3 * n] += [0, -1]

squares[3 * n:4 * n] += [1, 0]
squares[3 * n:4 * n, 1] *= -1
squares[3 * n:4 * n] += [0, 1]

squares[4 * n:5 * n] += [-1, 0]
squares[4 * n:5 * n, 1] *= -1
squares[4 * n:5 * n] += [0, 1]

squares[5 * n:6 * n] = squares[3 * n:4 * n] + [0, 1]
squares[6 * n:7 * n] = squares[3 * n:4 * n] + [0, -1]

squares[7 * n:8 * n] = squares[4 * n:5 * n] + [0, 1]
squares[8 * n:9 * n] = squares[4 * n:5 * n] + [0, -1]

In [ ]:
@njit
def compute_flat_kb_db():
    dm_condensed = np.empty((n * (n - 1)) // 2, dtype=np.float64)
    k = 0
    for i in range(n):
        x = square[i]
        for j in range(i + 1, n):
            sq_dists = np.sum((squares[j::n, :] - x) ** 2, axis=1)
            dm_condensed[k] = np.min(sq_dists)
            k += 1
    return np.sqrt(dm_condensed).astype(np.float32)

In [ ]:
dm = squareform(compute_flat_kb_db())

# Compute the regular persistent homology barcode

In [ ]:
VR = VietorisRipsPersistence(homology_dimensions=(0, 1, 2), metric="precomputed")
diagram = VR.fit_transform_plot([dm])[0];

# Steenrod barcodes

We use the ``rips_barcodes`` utility function for computing ordinary and Steenrod barcodes of (thresholded) Rips filtrations, starting from a distance matrix in this case.

## Persistent relative cohomology with a threshold at $R = 0.3$

In [ ]:
max_edge_length = 0.3
barcodes_kwargs = {
    "distance_matrix": True,
    "max_edge_length": max_edge_length,
    "k": 1,  # Compute Sq^1
    "max_simplex_dimension": 3,
    "absolute": False
}

barcode, st_barcode = rips_barcodes(dm, **barcodes_kwargs, verbose=True)

We can visualize the results either in diagram form, as multi-sets of points on a birth-death plane, or in barcode form.

The diagram visualization is as follows:

In [ ]:
plot_diagrams(barcode, st_barcode, k=1, kind="R")  # "R" for relative cohomology

A barcode visualization is as follows:

In [ ]:
n_dims = len(barcode)

lifetime_thresh = 0.
eps = 0.01
min_filtration_value = np.min(dm)

fig, (ax_rel_coho, ax_st) = plt.subplots(2, 1,
                                         figsize=(16, 8),
                                         sharex='col',
                                         gridspec_kw={'height_ratios': [2, 1]},
                                         tight_layout=True)

colors = ["Orange", "Green", "Blue", "Red"]
labels_rel_coho = [r"$\mathcal{H}^0_R$",
                   r"$\mathcal{H}^1_R$",
                   r"$\mathcal{H}^2_R$",
                   r"$\mathcal{H}^3_R$"]
labels_st = [r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^0_R$",
             r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^1_R$",
             r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^2_R$",
             r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^3_R$"]

counter = 0
for dim in range(1, n_dims):
    segs = []
    multiplicities = {}
    dgm = barcode[dim]
    dgm = dgm[dgm[:, 1] - dgm[:, 0] > lifetime_thresh]
    for p in dgm:
        if tuple(p) in multiplicities:
            multiplicities[tuple(p)] += 1
        else:
             multiplicities[tuple(p)] = 1

    counter_now = counter
    for i, (k, v) in enumerate(multiplicities.items()):
        death, birth = k
        y = - (counter_now + i)
        if death == -np.inf:
            ax_rel_coho.arrow(min_filtration_value - eps, y, -0.0000001, 0,
                              head_starts_at_zero=False, width=0, head_width=0.12, head_length=0.005,
                              color=colors[dim], ec=colors[dim])
            death = min_filtration_value - eps
        segs.append([[birth, y], [death, y]])
        if v > 1:
            ax_rel_coho.annotate(f"{v}", (death, y + 0.2))
        counter += 1

    segs = np.array(segs, dtype=np.float64)
    if len(segs):
        line_segments = LineCollection(segs, linewidths=2,
                                       colors=colors[dim],
                                       label=labels_rel_coho[dim],
                                       linestyle="solid")
        ax_rel_coho.add_collection(line_segments)

    counter += 1

ax_rel_coho.axvline(x=max_edge_length, color="gray", alpha=0.3)
ax_rel_coho.text(max_edge_length, y - 2/counter, rf"thresh = {max_edge_length}", rotation=90, fontdict={"fontsize": 15})

ax_rel_coho.autoscale()
ax_rel_coho.get_yaxis().set_visible(False)
ax_rel_coho.legend(loc="upper right", fontsize=18)
ax_rel_coho.margins(y=1/counter)
ax_rel_coho.set_title("Persistent relative cohomology barcode", fontdict={"fontsize": 22}, pad=15)

counter = 0
for dim in range(n_dims):
    segs = []
    multiplicities = {}
    dgm = st_barcode[dim]
    dgm = dgm[dgm[:, 1] - dgm[:, 0] > lifetime_thresh]
    for p in dgm:
        if tuple(p) in multiplicities:
            multiplicities[tuple(p)] += 1
        else:
             multiplicities[tuple(p)] = 1

    counter_now = counter
    for i, (k, v) in enumerate(multiplicities.items()):
        death, birth = k
        y = - (counter_now + i)
        if death == -np.inf:
            ax_st.arrow(min_filtration_value - eps, y, -0.0000001, 0,
                        head_starts_at_zero=False, width=0, head_width=0.1, head_length=0.005,
                        color=colors[dim], ec=colors[dim])
            death = min_filtration_value - eps
        segs.append([[birth, y], [death, y]])
        if v > 1:
            ax_st.annotate(f"{v}", (death, y + 0.2))
        counter += 1

    segs = np.array(segs, dtype=np.float64)
    if len(segs):
        line_segments = LineCollection(segs, linewidths=2,
                                       colors=colors[dim],
                                       label=labels_st[dim],
                                       linestyle="dashed")
        ax_st.add_collection(line_segments)

    counter += 1

ax_st.axvline(x=max_edge_length, color="gray", alpha=0.3)
ax_st.text(max_edge_length, y - 0.8, rf"thresh = {max_edge_length}", rotation=90, fontdict={"fontsize": 15})

ax_st.tick_params(axis="x", labelsize=18) 

ax_st.autoscale()
ax_st.get_yaxis().set_visible(False)
ax_st.legend(loc="upper right", fontsize=18)
ax_st.margins(y=10)
ax_st.set_title("Steenrod barcode", fontdict={"fontsize": 22}, pad=15)

# plt.savefig("flat_Klein_bottle_thresh_relative_barcodes.pdf")
plt.show()

# Persistent absolute cohomology, no threshold

In [ ]:
max_edge_length = np.inf
barcodes_kwargs["absolute"] = True
barcodes_kwargs["max_edge_length"] = max_edge_length

barcode, st_barcode = rips_barcodes(dm, **barcodes_kwargs, verbose=True)

Diagram visualization:

In [ ]:
plot_diagrams(barcode[:3], st_barcode[:3], k=1, kind="A")  # "A" for absolute cohomology

Barcode visualization:

In [ ]:
n_dims = len(barcode)

lifetime_thresh = 0.
eps = 0.01
max_filtration_value = np.max(dm)

fig, (ax_rel_coho, ax_st) = plt.subplots(2, 1,
                                         figsize=(16, 8),
                                         sharex='col',
                                         gridspec_kw={'height_ratios': [2, 1]},
                                         tight_layout=True)

colors = ["Orange", "Green", "Blue", "Red"]
labels_rel_coho = [r"$\mathcal{H}^0_A$",
                   r"$\mathcal{H}^1_A$",
                   r"$\mathcal{H}^2_A$",
                   r"$\mathcal{H}^3_A$"]
labels_st = [r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^0_A$",
             r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^1_A$",
             r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^2_A$",
             r"$\mathrm{img}(Sq^1) \cap \mathcal{H}^3_A$"]

counter = 0
for dim in range(1, n_dims - 1):
    segs = []
    multiplicities = {}
    dgm = barcode[dim]
    dgm = dgm[dgm[:, 1] - dgm[:, 0] > lifetime_thresh]
    for p in dgm:
        if tuple(p) in multiplicities:
            multiplicities[tuple(p)] += 1
        else:
             multiplicities[tuple(p)] = 1

    counter_now = counter
    for i, (k, v) in enumerate(multiplicities.items()):
        birth, death = k
        y = - (counter_now + i)
        if death == np.inf:
            ax_rel_coho.arrow(max_filtration_value + eps, y, 0.0000001, 0,
                              head_starts_at_zero=False, width=0, head_width=0.22, head_length=0.01,
                              color=colors[dim], ec=colors[dim])
            death = max_filtration_value + eps
        segs.append([[birth, y], [death, y]])
        if v > 1:
            ax_rel_coho.annotate(f"{v}", (death, y + 0.2))
        counter += 1

    segs = np.array(segs, dtype=np.float64)
    if len(segs):
        line_segments = LineCollection(segs, linewidths=2,
                                       colors=colors[dim],
                                       label=labels_rel_coho[dim],
                                       linestyle="solid")
        ax_rel_coho.add_collection(line_segments)

    counter += 1

# ax_rel_coho.axvline(x=max_edge_length, color="gray", alpha=0.3)
# ax_rel_coho.text(max_edge_length, y + 4, rf"thresh = {max_edge_length}", rotation=90, fontdict={"fontsize": 15})

ax_rel_coho.autoscale()
ax_rel_coho.get_yaxis().set_visible(False)
ax_rel_coho.legend(loc="lower left", fontsize=18)
ax_rel_coho.margins(y=1/counter)
ax_rel_coho.set_title("Persistent absolute cohomology barcode", fontdict={"fontsize": 22}, pad=15)

counter = 0
for dim in range(n_dims):
    segs = []
    multiplicities = {}
    dgm = st_barcode[dim]
    dgm = dgm[dgm[:, 1] - dgm[:, 0] > lifetime_thresh]
    for p in dgm:
        if tuple(p) in multiplicities:
            multiplicities[tuple(p)] += 1
        else:
             multiplicities[tuple(p)] = 1

    counter_now = counter
    for i, (k, v) in enumerate(multiplicities.items()):
        birth, death = k
        y = - (counter_now + i)
        if death == np.inf:
            ax_st.arrow(max_filtration_value + eps, y, 0.0000001, 0,
                        head_starts_at_zero=False, width=0, head_width=0.3, head_length=0.05,
                        color=colors[dim], ec=colors[dim])
            death = max_filtration_value + eps
        segs.append([[birth, y], [death, y]])
        if v > 1:
            ax_st.annotate(f"{v}", (death, y + 0.2))
        counter += 1

    segs = np.array(segs, dtype=np.float64)
    if len(segs):
        line_segments = LineCollection(segs, linewidths=2,
                                       colors=colors[dim],
                                       label=labels_st[dim],
                                       linestyle="dashed")
        ax_st.add_collection(line_segments)

    counter += 1

# ax_st.axvline(x=max_edge_length, color="gray", alpha=0.3)
# ax_st.text(max_edge_length, y - 0.13, rf"thresh = {max_edge_length}", rotation=90, fontdict={"fontsize": 15})

ax_st.tick_params(axis="x", labelsize=18) 

ax_st.autoscale()
ax_st.get_yaxis().set_visible(False)
ax_st.legend(loc="lower left", fontsize=18)
# ax_st.margins(y=1)
ax_st.set_title("Steenrod barcode", fontdict={"fontsize": 22}, pad=15)

# plt.savefig("flat_Klein_bottle_no_thresh_absolute_barcodes.pdf")
plt.show()